In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("Womens Clothing E-Commerce Reviews.csv")

In [ ]:
df

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...,...
23481,23481,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
23482,23482,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
23483,23483,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses
23484,23484,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses


In [ ]:
df=df.drop(columns="Unnamed: 0")

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
import re
import string
from nltk.corpus import stopwords

# Step 1: Data Preprocessing (Same as before)
def preprocess_text(text):
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = text.lower()  # Convert to lowercase
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

df['cleaned_text'] = df['Review Text'].apply(lambda x: preprocess_text(str(x)))

# Step 2: Create sentiment based on Rating
df['Sentiment'] = df['Rating'].apply(lambda x: 'positive' if x > 3 else ('neutral' if x == 3 else 'negative'))

# Step 3: Train-Test Split
X = df['cleaned_text']  # Feature
y = df['Sentiment']     # Target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Tokenization and Padding
# Convert text into sequences of tokens (integers)
tokenizer = Tokenizer(num_words=20000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad sequences to ensure they have the same length
max_sequence_length = 200  # You can adjust this value based on text length
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post', truncating='post')

# Step 5: Label Encoding (Convert target labels into numeric form)
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Step 6: Build the LSTM Model
embedding_dim = 128  # Dimension of the embedding layer

model = Sequential()
model.add(Embedding(input_dim=20000, output_dim=embedding_dim, input_length=max_sequence_length))  # Embedding layer
model.add(SpatialDropout1D(0.2))  # Add dropout to prevent overfitting
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))  # LSTM layer with dropout
model.add(Dense(3, activation='softmax'))  # 3 output units (positive, neutral, negative)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 7: Train the Model
history = model.fit(X_train_padded, y_train_encoded, epochs=5, batch_size=64, validation_data=(X_test_padded, y_test_encoded), verbose=2)

# Step 8: Evaluate the Model
loss, accuracy = model.evaluate(X_test_padded, y_test_encoded, verbose=2)
print(f"Test Accuracy: {accuracy}")

# Step 9: Classification Report
from sklearn.metrics import classification_report

y_pred = model.predict(X_test_padded)
y_pred_classes = y_pred.argmax(axis=-1)  # Get predicted class labels
print("Classification Report:\n", classification_report(y_test_encoded, y_pred_classes, target_names=label_encoder.classes_))


Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


294/294 - 149s - 506ms/step - accuracy: 0.7766 - loss: 0.6932 - val_accuracy: 0.7680 - val_loss: 0.7019
Epoch 2/5
294/294 - 201s - 684ms/step - accuracy: 0.7771 - loss: 0.6871 - val_accuracy: 0.7680 - val_loss: 0.7007
Epoch 3/5
294/294 - 202s - 686ms/step - accuracy: 0.7771 - loss: 0.6864 - val_accuracy: 0.7680 - val_loss: 0.7019
Epoch 4/5
294/294 - 210s - 713ms/step - accuracy: 0.7771 - loss: 0.6857 - val_accuracy: 0.7680 - val_loss: 0.7023
Epoch 5/5
294/294 - 195s - 664ms/step - accuracy: 0.7771 - loss: 0.6855 - val_accuracy: 0.7680 - val_loss: 0.7063
147/147 - 14s - 98ms/step - accuracy: 0.7680 - loss: 0.7063
Test Accuracy: 0.7679863572120667
147/147 ━━━━━━━━━━━━━━━━━━━━ 15s 104ms/step
Classification Report:
               precision    recall  f1-score   support

    negative       0.00      0.00      0.00       470
     neutral       0.00      0.00      0.00       620
    positive       0.77      1.00      0.87      3608

    accuracy                           0.77      4698
   mac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
